# W210 Precipitation Data Cleaning

#### Import Packages

In [1]:
import pandas as pd
import numpy as np
import netCDF4 as ncdf
import os
import datetime
from math import pi
import requests

#### Test Out the Data Pull

In [2]:
# read through the text file with all the download links
file_text = open('/Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/subset_M2T1NXFLX_5.12.4_20221113_2020data.txt', 'r')
Lines = file_text.readlines()
# len(Lines)
# Lines[1:]
Links_2020 = Lines[1:]
len(Links_2020)

366

In [3]:
Links_2020[0:4]

['https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2T1NXFLX.5.12.4%2F2020%2F01%2FMERRA2_400.tavg1_2d_flx_Nx.20200101.nc4&SHORTNAME=M2T1NXFLX&FORMAT=bmM0Lw&LABEL=MERRA2_400.tavg1_2d_flx_Nx.20200101.SUB.nc&DATASET_VERSION=5.12.4&SERVICE=L34RS_MERRA2&VARIABLES=PRECTOTCORR%2CQLML%2CQSH%2CQSTAR%2CRHOA%2CSPEED%2CTLML%2CULML%2CVLML&VERSION=1.02&BBOX=-34%2C-74%2C6%2C-34\n',
 'https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2T1NXFLX.5.12.4%2F2020%2F01%2FMERRA2_400.tavg1_2d_flx_Nx.20200102.nc4&SHORTNAME=M2T1NXFLX&FORMAT=bmM0Lw&LABEL=MERRA2_400.tavg1_2d_flx_Nx.20200102.SUB.nc&DATASET_VERSION=5.12.4&SERVICE=L34RS_MERRA2&VARIABLES=PRECTOTCORR%2CQLML%2CQSH%2CQSTAR%2CRHOA%2CSPEED%2CTLML%2CULML%2CVLML&VERSION=1.02&BBOX=-34%2C-74%2C6%2C-34\n',
 'https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2T1NXFLX.5.12.4%2F2020%2F01%2FMERRA2_400.tavg1_2d_flx_N

In [4]:
# Extract date information from the link
example_str = Links_2020[0]
example_str.split("flx_Nx.")[1][:8]

'20200101'

In [5]:
!pwd

/Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon


In [8]:
!mkdir precip_data

In [49]:
# create .nc4 filenames and write the GET request result into the .nc4 file
result_20170101_test = requests.get(str(Links_2017[0]))
filename_20170101 = "/Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_" + "20170101" + ".nc4"
result_20170101_test.raise_for_status()
f = open(filename_20170101,'wb')
f.write(result_20170101_test.content)
f.close()

# read the .nc4 file to data frame
data_20170101_test = ncdf.Dataset(filename_20170101, mode='r')
# data_20170101_test

NameError: name 'Links_2017' is not defined

In [6]:
result_20200101_test = requests.get(str(Links_2020[0]))
filename_20200101 = "/Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_" + "20200101" + ".nc4"
result_20200101_test.raise_for_status()
f = open(filename_20200101,'wb')
f.write(result_20200101_test.content)
f.close()

# read the .nc4 file to data frame
data_20200101_test = ncdf.Dataset(filename_20200101, mode='r')
# data_20200101_test

In [7]:
data_20200101_test

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    CDI: Climate Data Interface version 1.9.8 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1
    History: Original file generated: Mon Jan 13 21:32:58 2020 GMT
    Comment: GMAO filename: d5124_m2_jan10.tavg1_2d_flx_Nx.20200101.nc4
    Filename: MERRA2_400.tavg1_2d_flx_Nx.20200101.nc4
    Institution: NASA Global Modeling and Assimilation Office
    References: http://gmao.gsfc.nasa.gov
    Format: NetCDF-4/HDF-5
    SpatialCoverage: global
    VersionID: 5.12.4
    TemporalRange: 1980-01-01 -> 2016-12-31
    identifier_product_doi_authority: http://dx.doi.org/
    ShortName: M2T1NXFLX
    GranuleID: MERRA2_400.tavg1_2d_flx_Nx.20200101.nc4
    ProductionDateTime: Original file generated: Mon Jan 13 21:32:58 2020 GMT
    LongName: MERRA2 tavg1_2d_flx_Nx: 2d,1-Hourly,Time-Averaged,Single-Level,Assimilation,Surface Flux Diagnostics
    Title: MERRA2 tavg1_2d_flx_Nx: 2d,1-Hourly,Time-Averaged,Single-Le

#### Key Variable Lookup
* PRECTOTCORR = bias corrected total precipitation
* QLML = surface specific humidity
* QSH = effective surface specific humidity
* QSTAR = surface moisture scale
* RHOA = air density at surface
* SPEED = surface wind speed
* TLML = surface air temperature
* ULML = surface eastward wind
* VLML = surface northward wind
* time
* lon
* lat

In [23]:
data_20200101_test['time'][:]

masked_array(data=[   0,   60,  120,  180,  240,  300,  360,  420,  480,
                    540,  600,  660,  720,  780,  840,  900,  960, 1020,
                   1080, 1140, 1200, 1260, 1320, 1380],
             mask=False,
       fill_value=999999,
            dtype=int32)

In [10]:
lats = [x/10 for x in range(-340, 65, 5)]
lons = [x/1000 for x in range(-73750, -33875, 625)]

In [11]:
print("length of lats is ", len(lats))
print("length of lons is ", len(lons))

length of lats is  81
length of lons is  64


In [25]:
times = list(data_20170101_test['time'][:])

In [33]:
len(data_20170101_test['PRECTOTCORR'][:,:,:])

24

In [34]:
pd.DataFrame(data_20170101_test['PRECTOTCORR'][:,:,:][1]).reset_index().melt('index')

,index,variable,value
0,0,0,1.602457e-07
1,1,0,1.740991e-07
2,2,0,6.865594e-08
3,3,0,1.988519e-08
4,4,0,9.215000e-09
...,...,...,...
5179,76,63,1.394153e-04
5180,77,63,1.307726e-04
5181,78,63,1.173317e-04
5182,79,63,6.800890e-05


In [41]:
pd.DataFrame(data_20170101_test['QLML'][:,:,:][1]).reset_index().melt('index')

,index,variable,value
0,0,0,0.009941
1,1,0,0.009735
2,2,0,0.009342
3,3,0,0.008960
4,4,0,0.008811
...,...,...,...
5179,76,63,0.017765
5180,77,63,0.017490
5181,78,63,0.017047
5182,79,63,0.016704


In [48]:
pd.DataFrame(data_20170101_test['QSH'][:,:,:][1]).reset_index().melt('index')

,index,variable,value
0,0,0,0.012298
1,1,0,0.012569
2,2,0,0.012813
3,3,0,0.012904
4,4,0,0.013022
...,...,...,...
5179,76,63,0.022738
5180,77,63,0.022639
5181,78,63,0.022532
5182,79,63,0.022464


In [50]:
pd.DataFrame(data_20170101_test['QSTAR'][:,:,:][23]).reset_index().melt('index')

,index,variable,value
0,0,0,0.000112
1,1,0,0.000116
2,2,0,0.000120
3,3,0,0.000121
4,4,0,0.000122
...,...,...,...
5179,76,63,0.000155
5180,77,63,0.000150
5181,78,63,0.000146
5182,79,63,0.000159


In [52]:
pd.DataFrame(data_20170101_test['RHOA'][:,:,:][1]).reset_index().melt('index')

,index,variable,value
0,0,0,1.220330
1,1,0,1.219720
2,2,0,1.218865
3,3,0,1.218011
4,4,0,1.217278
...,...,...,...
5179,76,63,1.163934
5180,77,63,1.164910
5181,78,63,1.166131
5182,79,63,1.167596


In [53]:
pd.DataFrame(data_20170101_test['SPEED'][:,:,:][1]).reset_index().melt('index')

,index,variable,value
0,0,0,6.597205
1,1,0,7.274940
2,2,0,8.269080
3,3,0,9.159705
4,4,0,9.874549
...,...,...,...
5179,76,63,6.485877
5180,77,63,7.095252
5181,78,63,7.458533
5182,79,63,7.485877


In [54]:
pd.DataFrame(data_20170101_test['TLML'][:,:,:][1]).reset_index().melt('index')

,index,variable,value
0,0,0,289.185547
1,1,0,289.357422
2,2,0,289.560547
3,3,0,289.771484
4,4,0,289.896484
...,...,...,...
5179,76,63,299.724609
5180,77,63,299.552734
5181,78,63,299.365234
5182,79,63,299.099609


In [60]:
PRECTOTCORR_cube = data_20170101_test['PRECTOTCORR'][:,:,:]
QLML_cube = data_20170101_test['QLML'][:,:,:]
QSH_cube = data_20170101_test['QSH'][:,:,:]

melted_PRECTOTCORR = pd.DataFrame(PRECTOTCORR_cube[1]).reset_index().melt('index')
melted_PRECTOTCORR.columns = ['lat_key', 'lon_key', 'PRECTOTCORR']
melted_PRECTOTCORR['lat'] = melted_PRECTOTCORR['lat_key'].apply(lambda x: lats[x])
melted_PRECTOTCORR['lon'] = melted_PRECTOTCORR['lon_key'].apply(lambda x: lons[x])
    
melted_QLML = pd.DataFrame(QLML_cube[1]).reset_index().melt('index')
melted_QLML.columns = ['lat_key', 'lon_key', 'QLML']
melted_QLML['lat'] = melted_QLML['lat_key'].apply(lambda x: lats[x])
melted_QLML['lon'] = melted_QLML['lon_key'].apply(lambda x: lons[x])

melted_QSH = pd.DataFrame(QSH_cube[1]).reset_index().melt('index')
melted_QSH.columns = ['lat_key', 'lon_key', 'QSH']
melted_QSH['lat'] = melted_QSH['lat_key'].apply(lambda x: lats[x])
melted_QSH['lon'] = melted_QSH['lon_key'].apply(lambda x: lons[x])

In [63]:
dfs_merged = pd.merge(melted_PRECTOTCORR, melted_QLML)[['lat','lon','PRECTOTCORR','QLML']]
dfs_merged = pd.merge(dfs_merged, melted_QSH)[['lat','lon','PRECTOTCORR','QLML', 'QSH']]
dfs_merged

,lat,lon,PRECTOTCORR,QLML,QSH
0,-34.0,-73.750,1.602457e-07,0.009941,0.012298
1,-33.5,-73.750,1.740991e-07,0.009735,0.012569
2,-33.0,-73.750,6.865594e-08,0.009342,0.012813
3,-32.5,-73.750,1.988519e-08,0.008960,0.012904
4,-32.0,-73.750,9.215000e-09,0.008811,0.013022
...,...,...,...,...,...
5179,4.0,-34.375,1.394153e-04,0.017765,0.022738
5180,4.5,-34.375,1.307726e-04,0.017490,0.022639
5181,5.0,-34.375,1.173317e-04,0.017047,0.022532
5182,5.5,-34.375,6.800890e-05,0.016704,0.022464


In [38]:
# read through the text file with all the download links
file_text = open('/Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/subset_M2T1NXFLX_5.12.4_20221106_2020data.txt', 'r')
Lines = file_text.readlines()
# len(Lines)
# Lines[1:]
Links_2020 = Lines[1:]
len(Links_2020)

366

In [52]:
# define the function to clean and process the precipitation data

def read_clean_precip_2021():
    
    df = pd.DataFrame()
    for i in range(len(Links_2021)):
        url_i = str(Links_2021[i])
        date_i = url_i.split("flx_Nx.")[1][:8]
        FILENAME = "/Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_" + str(date_i) + ".nc4"
        result = requests.get(url_i)
        try:
            result.raise_for_status()
            f = open(FILENAME,'wb')
            f.write(result.content)
            f.close()
            print('contents of URL written to '+FILENAME)
        except:
            print('requests.get() returned an error code '+str(result.status_code))
        
        data = ncdf.Dataset(FILENAME, mode='r') # read .nc4 file

        PRECTOTCORR_cube = data['PRECTOTCORR'][:,:,:]
        QLML_cube = data['QLML'][:,:,:]
        QSH_cube = data['QSH'][:,:,:]
        QSTAR_cube = data['QSTAR'][:,:,:]
        RHOA_cube = data['RHOA'][:,:,:]
        SPEED_cube = data['SPEED'][:,:,:]
        TLML_cube = data['TLML'][:,:,:]
        ULML_cube = data['ULML'][:,:,:]
        VLML_cube = data['VLML'][:,:,:]
        times = list(data['time'][:])
        
        dfs = []
        for x in range(0, len(times)):
            melted_PRECTOTCORR = pd.DataFrame(PRECTOTCORR_cube[x]).reset_index().melt('index')
            melted_PRECTOTCORR.columns = ['lat_key', 'lon_key', 'PRECTOTCORR']
            melted_PRECTOTCORR['lat'] = melted_PRECTOTCORR['lat_key'].apply(lambda x: lats[x])
            melted_PRECTOTCORR['lon'] = melted_PRECTOTCORR['lon_key'].apply(lambda x: lons[x])

            melted_QLML = pd.DataFrame(QLML_cube[x]).reset_index().melt('index')
            melted_QLML.columns = ['lat_key', 'lon_key', 'QLML']
            melted_QLML['lat'] = melted_QLML['lat_key'].apply(lambda x: lats[x])
            melted_QLML['lon'] = melted_QLML['lon_key'].apply(lambda x: lons[x])

            melted_QSH = pd.DataFrame(QSH_cube[x]).reset_index().melt('index')
            melted_QSH.columns = ['lat_key', 'lon_key', 'QSH']
            melted_QSH['lat'] = melted_QSH['lat_key'].apply(lambda x: lats[x])
            melted_QSH['lon'] = melted_QSH['lon_key'].apply(lambda x: lons[x])

            melted_QSTAR = pd.DataFrame(QSTAR_cube[x]).reset_index().melt('index')
            melted_QSTAR.columns = ['lat_key', 'lon_key', 'QSTAR']
            melted_QSTAR['lat'] = melted_QSTAR['lat_key'].apply(lambda x: lats[x])
            melted_QSTAR['lon'] = melted_QSTAR['lon_key'].apply(lambda x: lons[x])

            melted_RHOA = pd.DataFrame(RHOA_cube[x]).reset_index().melt('index')
            melted_RHOA.columns = ['lat_key', 'lon_key', 'RHOA']
            melted_RHOA['lat'] = melted_RHOA['lat_key'].apply(lambda x: lats[x])
            melted_RHOA['lon'] = melted_RHOA['lon_key'].apply(lambda x: lons[x])

            melted_SPEED = pd.DataFrame(SPEED_cube[x]).reset_index().melt('index')
            melted_SPEED.columns = ['lat_key', 'lon_key', 'SPEED']
            melted_SPEED['lat'] = melted_SPEED['lat_key'].apply(lambda x: lats[x])
            melted_SPEED['lon'] = melted_SPEED['lon_key'].apply(lambda x: lons[x])

            melted_TLML = pd.DataFrame(TLML_cube[x]).reset_index().melt('index')
            melted_TLML.columns = ['lat_key', 'lon_key', 'TLML']
            melted_TLML['lat'] = melted_TLML['lat_key'].apply(lambda x: lats[x])
            melted_TLML['lon'] = melted_TLML['lon_key'].apply(lambda x: lons[x])

            melted_ULML = pd.DataFrame(ULML_cube[x]).reset_index().melt('index')
            melted_ULML.columns = ['lat_key', 'lon_key', 'ULML']
            melted_ULML['lat'] = melted_ULML['lat_key'].apply(lambda x: lats[x])
            melted_ULML['lon'] = melted_ULML['lon_key'].apply(lambda x: lons[x])

            melted_VLML = pd.DataFrame(VLML_cube[x]).reset_index().melt('index')
            melted_VLML.columns = ['lat_key', 'lon_key', 'VLML']
            melted_VLML['lat'] = melted_VLML['lat_key'].apply(lambda x: lats[x])
            melted_VLML['lon'] = melted_VLML['lon_key'].apply(lambda x: lons[x])

            dfs_merged = pd.merge(melted_PRECTOTCORR, melted_QLML)[['lat','lon','PRECTOTCORR','QLML']]
            dfs_merged = pd.merge(dfs_merged, melted_QSH)[['lat','lon','PRECTOTCORR','QLML', 'QSH']]
            dfs_merged = pd.merge(dfs_merged, melted_QSTAR)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR']]
            dfs_merged = pd.merge(dfs_merged, melted_RHOA)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA']]
            dfs_merged = pd.merge(dfs_merged, melted_SPEED)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA', 'SPEED']]
            dfs_merged = pd.merge(dfs_merged, melted_TLML)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA', 'SPEED', 'TLML']]
            dfs_merged = pd.merge(dfs_merged, melted_ULML)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA', 'SPEED', 'TLML', 'ULML']]
            dfs_merged = pd.merge(dfs_merged, melted_VLML)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA', 'SPEED', 'TLML', 'ULML', 'VLML']]

            dfs.append(dfs_merged)
            dfs[x]['Time_Hour'] = times[x] / 60
            merged_df = pd.concat(dfs)
            merged_df['date'] = date_i
        df = pd.concat([df, merged_df], axis=0)
    return(df)

In [8]:
# 2020 data doesn't have TLML and ULML variables, so need to redefine the function

def read_clean_precip_2020():
    
    df = pd.DataFrame()
    for i in range(len(Links_2020)):
        url_i = str(Links_2020[i])
        date_i = url_i.split("flx_Nx.")[1][:8]
        FILENAME = "/Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_" + str(date_i) + ".nc4"
        result = requests.get(url_i)
        try:
            result.raise_for_status()
            f = open(FILENAME,'wb')
            f.write(result.content)
            f.close()
            print('contents of URL written to '+FILENAME)
        except:
            print('requests.get() returned an error code '+str(result.status_code))
        
        data = ncdf.Dataset(FILENAME, mode='r') # read .nc4 file

        PRECTOTCORR_cube = data['PRECTOTCORR'][:,:,:]
        QLML_cube = data['QLML'][:,:,:]
        QSH_cube = data['QSH'][:,:,:]
        QSTAR_cube = data['QSTAR'][:,:,:]
        RHOA_cube = data['RHOA'][:,:,:]
        SPEED_cube = data['SPEED'][:,:,:]
        TLML_cube = data['TLML'][:,:,:]
        ULML_cube = data['ULML'][:,:,:]
        VLML_cube = data['VLML'][:,:,:]
        times = list(data['time'][:])
        
        dfs = []
        for x in range(0, len(times)):
            melted_PRECTOTCORR = pd.DataFrame(PRECTOTCORR_cube[x]).reset_index().melt('index')
            melted_PRECTOTCORR.columns = ['lat_key', 'lon_key', 'PRECTOTCORR']
            melted_PRECTOTCORR['lat'] = melted_PRECTOTCORR['lat_key'].apply(lambda x: lats[x])
            melted_PRECTOTCORR['lon'] = melted_PRECTOTCORR['lon_key'].apply(lambda x: lons[x])

            melted_QLML = pd.DataFrame(QLML_cube[x]).reset_index().melt('index')
            melted_QLML.columns = ['lat_key', 'lon_key', 'QLML']
            melted_QLML['lat'] = melted_QLML['lat_key'].apply(lambda x: lats[x])
            melted_QLML['lon'] = melted_QLML['lon_key'].apply(lambda x: lons[x])

            melted_QSH = pd.DataFrame(QSH_cube[x]).reset_index().melt('index')
            melted_QSH.columns = ['lat_key', 'lon_key', 'QSH']
            melted_QSH['lat'] = melted_QSH['lat_key'].apply(lambda x: lats[x])
            melted_QSH['lon'] = melted_QSH['lon_key'].apply(lambda x: lons[x])

            melted_QSTAR = pd.DataFrame(QSTAR_cube[x]).reset_index().melt('index')
            melted_QSTAR.columns = ['lat_key', 'lon_key', 'QSTAR']
            melted_QSTAR['lat'] = melted_QSTAR['lat_key'].apply(lambda x: lats[x])
            melted_QSTAR['lon'] = melted_QSTAR['lon_key'].apply(lambda x: lons[x])

            melted_RHOA = pd.DataFrame(RHOA_cube[x]).reset_index().melt('index')
            melted_RHOA.columns = ['lat_key', 'lon_key', 'RHOA']
            melted_RHOA['lat'] = melted_RHOA['lat_key'].apply(lambda x: lats[x])
            melted_RHOA['lon'] = melted_RHOA['lon_key'].apply(lambda x: lons[x])

            melted_SPEED = pd.DataFrame(SPEED_cube[x]).reset_index().melt('index')
            melted_SPEED.columns = ['lat_key', 'lon_key', 'SPEED']
            melted_SPEED['lat'] = melted_SPEED['lat_key'].apply(lambda x: lats[x])
            melted_SPEED['lon'] = melted_SPEED['lon_key'].apply(lambda x: lons[x])

            melted_TLML = pd.DataFrame(TLML_cube[x]).reset_index().melt('index')
            melted_TLML.columns = ['lat_key', 'lon_key', 'TLML']
            melted_TLML['lat'] = melted_TLML['lat_key'].apply(lambda x: lats[x])
            melted_TLML['lon'] = melted_TLML['lon_key'].apply(lambda x: lons[x])

            melted_ULML = pd.DataFrame(ULML_cube[x]).reset_index().melt('index')
            melted_ULML.columns = ['lat_key', 'lon_key', 'ULML']
            melted_ULML['lat'] = melted_ULML['lat_key'].apply(lambda x: lats[x])
            melted_ULML['lon'] = melted_ULML['lon_key'].apply(lambda x: lons[x])

            melted_VLML = pd.DataFrame(VLML_cube[x]).reset_index().melt('index')
            melted_VLML.columns = ['lat_key', 'lon_key', 'VLML']
            melted_VLML['lat'] = melted_VLML['lat_key'].apply(lambda x: lats[x])
            melted_VLML['lon'] = melted_VLML['lon_key'].apply(lambda x: lons[x])

            dfs_merged = pd.merge(melted_PRECTOTCORR, melted_QLML)[['lat','lon','PRECTOTCORR','QLML']]
            dfs_merged = pd.merge(dfs_merged, melted_QSH)[['lat','lon','PRECTOTCORR','QLML', 'QSH']]
            dfs_merged = pd.merge(dfs_merged, melted_QSTAR)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR']]
            dfs_merged = pd.merge(dfs_merged, melted_RHOA)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA']]
            dfs_merged = pd.merge(dfs_merged, melted_SPEED)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA', 'SPEED']]
            dfs_merged = pd.merge(dfs_merged, melted_TLML)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA', 'SPEED', 'TLML']]
            dfs_merged = pd.merge(dfs_merged, melted_ULML)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA', 'SPEED', 'TLML', 'ULML']]
            dfs_merged = pd.merge(dfs_merged, melted_VLML)[['lat','lon','PRECTOTCORR','QLML', 'QSH', 'QSTAR', 'RHOA', 'SPEED', 'TLML', 'ULML', 'VLML']]

            dfs.append(dfs_merged)
            dfs[x]['Time_Hour'] = times[x] / 60
            merged_df = pd.concat(dfs)
            merged_df['date'] = date_i
        df = pd.concat([df, merged_df], axis=0)
    return(df)

In [12]:
df_precip_2020 = read_clean_precip_2020()

contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200101.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200102.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200103.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200104.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200105.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200106.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200107.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200108.nc4
contents of URL written 

contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200309.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200310.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200311.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200312.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200313.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200314.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200315.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200316.nc4
contents of URL written 

contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200516.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200517.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200518.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200519.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200520.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200521.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200522.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200523.nc4
contents of URL written 

contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200723.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200724.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200725.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200726.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200727.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200728.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200729.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200730.nc4
contents of URL written 

contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200929.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20200930.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201001.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201002.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201003.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201004.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201005.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201006.nc4
contents of URL written 

contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201206.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201207.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201208.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201209.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201210.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201211.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201212.nc4
contents of URL written to /Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/amazon/precip_data/precip_20201213.nc4
contents of URL written 

In [13]:
df_precip_2020

,lat,lon,PRECTOTCORR,QLML,QSH,QSTAR,RHOA,SPEED,TLML,ULML,VLML,Time_Hour,date
0,-34.0,-73.750,0.000000,0.009202,0.011638,0.000064,1.217502,10.974510,288.589752,4.522119,9.998275,0.0,20200101
1,-33.5,-73.750,0.000000,0.009225,0.011722,0.000066,1.216709,10.361229,288.847565,3.521142,9.740462,0.0,20200101
2,-33.0,-73.750,0.000000,0.009320,0.011570,0.000060,1.216281,9.833885,288.972565,2.715966,9.447494,0.0,20200101
3,-32.5,-73.750,0.000000,0.009480,0.011745,0.000061,1.216098,9.501854,289.019440,2.082910,9.263900,0.0,20200101
4,-32.0,-73.750,0.000000,0.009633,0.011959,0.000063,1.215549,9.439354,289.128815,1.550470,9.302962,0.0,20200101
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5179,4.0,-34.375,0.000186,0.018308,0.023520,0.000168,1.163816,3.735872,299.844177,-3.055999,2.146971,23.0,20201231
5180,4.5,-34.375,0.000232,0.018148,0.023696,0.000183,1.163999,3.259309,299.859802,-2.702483,1.810057,23.0,20201231
5181,5.0,-34.375,0.000297,0.018072,0.023718,0.000186,1.164366,3.338411,299.812927,-3.083343,1.127928,23.0,20201231
5182,5.5,-34.375,0.000317,0.018239,0.023619,0.000174,1.165403,3.933137,299.562927,-3.905608,0.218260,23.0,20201231


In [14]:
df_precip_2020.to_csv("/Users/ChangLiu/Documents/UC Berkeley MIDS/210 Capstone/precip_data/combined_precip_data_2020.csv")